<a href="https://colab.research.google.com/github/MohamedKhaledBenmna/100-Days-Of-ML-Code/blob/master/TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U pip
! pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 41.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.2/566.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.6/574.6 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    

In [ ]:
!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

--2022-11-17 08:27:55--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘tts_train_dir/LJSpeech-1.1.tar.bz2’

tts_train_dir/LJSpe 100%[===================>]   2.56G   144MB/s    in 19s     

2022-11-17 08:28:14 (136 MB/s) - ‘tts_train_dir/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [ ]:
import pandas as pd
df = pd.read_csv( "/content/tts_train_dir/LJSpeech-1.1/metadata.csv", on_bad_lines="skip", sep = "|")
df.head(10)
newdf= df.iloc[0:100, :]
from pathlib import Path  
filepath = Path('/content/tts_train_dir/LJSpeech-1.1/out.csv')  
newdf.to_csv("/content/tts_train_dir/LJSpeech-1.1/out.csv", index= False , sep = "|" )

dataset_config = BaseDatasetConfig(  meta_file_train="out.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
, formatter="ljspeech")


In [ ]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

In [ ]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:
from TTS.tts.datasets import *
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
     eval_split_size=config.eval_split_size,
)

 | > Found 101 files in /content/tts_train_dir/LJSpeech-1.1


In [ ]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [ ]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False

 > Model has 28610257 parameters


In [45]:
trainer.fit()


 > EPOCH: 0/100
 --> tts_train_dir/run-November-17-2022_09+42AM-0000000


[*] Pre-computing phonemes...


  2%|▏         | 2/100 [00:00<00:42,  2.32it/s]

fɚ ðə sɛvəntinθ sɛnt͡ʃɚi lɛtɚz ɑɹ æt list pjʊɹ ænd sɪmpəl ɪn laɪn. ðə ɪtæliən, boʊdəni, ænd ðə fɹɛnt͡ʃmæn, dɪdət,
 [!] Character '͡' not found in the vocabulary. Discarding it.


100%|██████████| 100/100 [00:07<00:00, 13.84it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

 > TRAINING (2022-11-17 09:42:28) 


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.36434 (+0.08383)
     | > avg_loss: 1.83170 (-0.04566)
     | > avg_log_mle: 0.43046 (-0.00401)
     | > avg_loss_dur: 1.40123 (-0.04165)

 > BEST MODEL : tts_train_dir/run-November-17-2022_09+42AM-0000000/best_model_396.pth

 > EPOCH: 99/100
 --> tts_train_dir/run-November-17-2022_09+42AM-0000000

 > TRAINING (2022-11-17 12:56:54) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 1 not found characters:
	| > ͡
| > Number of instances : 100
 | > Preprocessing samples
 | > Max text length: 169
 | > Min text length: 26
 | > Avg text length: 100.83
 | 
 | > Max audio length: 219315.0
 | > Min audio length: 39347.0
 | > Avg audio length: 145581.88
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 1 not found characters:
	| > ͡
| > Number of instances : 1
 | > Preprocessing samples
 | > Max text length: 140
 | > Min text length: 140
 | > Avg text length: 140.0
 | 
 | > Max audio length: 212659.0
 | > Min audio length: 212659.0
 | > Avg audio length: 212659.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.27882 (-0.08552)
     | > avg_loss: 1.81379 (-0.01791)
     | > avg_log_mle: 0.42697 (-0.00349)
     | > avg_loss_dur: 1.38682 (-0.01442)

 > BEST MODEL : tts_train_dir/run-November-17-2022_09+42AM-0000000/best_model_400.pth


In [46]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])


In [47]:
 !tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

usage: tts [-h] [--list_models [LIST_MODELS]]
           [--model_info_by_idx MODEL_INFO_BY_IDX]
           [--model_info_by_name MODEL_INFO_BY_NAME] [--text TEXT]
           [--model_name MODEL_NAME] [--vocoder_name VOCODER_NAME]
           [--config_path CONFIG_PATH] [--model_path MODEL_PATH]
           [--out_path OUT_PATH] [--use_cuda USE_CUDA]
           [--vocoder_path VOCODER_PATH]
           [--vocoder_config_path VOCODER_CONFIG_PATH]
           [--encoder_path ENCODER_PATH]
           [--encoder_config_path ENCODER_CONFIG_PATH]
           [--speakers_file_path SPEAKERS_FILE_PATH]
           [--language_ids_file_path LANGUAGE_IDS_FILE_PATH]
           [--speaker_idx SPEAKER_IDX] [--language_idx LANGUAGE_IDX]
           [--speaker_wav SPEAKER_WAV [SPEAKER_WAV ...]]
           [--gst_style GST_STYLE]
           [--capacitron_style_wav CAPACITRON_STYLE_WAV]
           [--capacitron_style_text CAPACITRON_STYLE_TEXT]
           [--list_speaker_idxs [LIST_SPEAKER_IDXS]]
           [-